In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# enter the foldername in your Drive where you have saved the unzipped
# assignment folder, e.g. 'cs231n/assignments/assignment3/'
FOLDERNAME = "Colab Notebooks/mango_class"
assert FOLDERNAME is not None, "[!] Enter the foldername."

# now that we've mounted your Drive, this ensures that
# the Python interpreter of the Colab VM can load
# python files from within it.
import sys
sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))

# this downloads the CIFAR-10 dataset to your Drive
# if it doesn't already exist.
%cd drive/My\ Drive/$FOLDERNAME/

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
/content/drive/My Drive/Colab Notebooks/mango_class


In [ ]:
import lightgbm as lgb


In [ ]:
import os
import pandas as pd
import pickle
import seaborn as sns
import matplotlib.image as mpimg
from PIL import Image#https://yungyuc.github.io/oldtech/python/python_imaging.html
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset, DataLoader
import torch.utils.data as data_utils
import glob
import os.path as osp
import numpy as np
from PIL import Image#https://yungyuc.github.io/oldtech/python/python_imaging.html
import torch.optim as optim
from torch.optim import lr_scheduler


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:

import torchvision.models as models
import torch.nn as nn
from sklearn.svm import LinearSVC as svm
from sklearn.model_selection import cross_validate
from collections import Counter
import time
from py_files.dataloader3 import *

In [ ]:
import torch
USE_GPU = True

# dtype = torch.float32 # we will be using float throughout this tutorial

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

# Constant to control how frequently we print train loss
print_every = 100

print('using device:', device)


using device: cuda


In [ ]:

X1 = torch.load(open('./dataset/train_x_batch5.pt', 'rb'))
y1 = torch.load(open('./dataset/train_y_batch5.pt', 'rb'))

X2 = torch.load(open('./dataset/train_x_batch6.pt', 'rb'))
y2 = torch.load(open('./dataset/train_y_batch6.pt', 'rb'))

X3 = torch.load(open('./dataset/train_x_batch7.pt', 'rb'))
y3 = torch.load(open('./dataset/train_y_batch7.pt', 'rb'))

X4 = torch.load(open('./dataset/train_x_batch8.pt', 'rb'))
y4 = torch.load(open('./dataset/train_y_batch8.pt', 'rb'))

dataset1 = data_utils.TensorDataset(X1, y1)
dataset2 = data_utils.TensorDataset(X2, y2)
dataset3 = data_utils.TensorDataset(X3, y3)
dataset4 = data_utils.TensorDataset(X4, y4)

In [ ]:
class ConcatDataset(Dataset):
    def __init__(self, *datasets):
        self.datasets = datasets

    def __getitem__(self, i):
        return tuple(d[i] for d in self.datasets)

    def __len__(self):
        return min(len(d) for d in self.datasets)

In [ ]:
metaset1_loader = DataLoader(
                    ConcatDataset(
                      dataset1),
                      batch_size=128, 
                      shuffle=False)
metaset2_loader = DataLoader(
                      ConcatDataset(
                      dataset2),
                      batch_size=128, 
                      shuffle=False)
metaset3_loader = DataLoader(
                      ConcatDataset(
                      dataset3),
                      batch_size=128, 
                      shuffle=False)
metaset4_loader = DataLoader(
                      ConcatDataset(
                      dataset4),
                      batch_size=128, 
                      shuffle=False)

In [ ]:

model5 = models.vgg16(pretrained=True)

for param in model5.features[0: 26].parameters():
    param.requires_grad = False
model5.avgpool =  nn.AdaptiveAvgPool2d(output_size=(1, 1))
model5.classifier = nn.Sequential(
    nn.Linear(in_features=512, out_features=512, bias=True),
    nn.ReLU(inplace=True),
    nn.Dropout(p=0.5, inplace=False),
    nn.Linear(in_features=512, out_features=128, bias=True),
    nn.ReLU(inplace=True),
    nn.Dropout(p=0.5, inplace=False),
    nn.Linear(in_features=128, out_features=3, bias=True),
)

model5.load_state_dict(torch.load("./model_params/" + "vgg" + "_cv_dev" + str(5) + ".pth"))

model5 = nn.Sequential(model5.features, model5.avgpool, nn.Flatten())
#detach scores from model1

model5 = model5.to(device)
###############################################################################
model6 = models.squeezenet1_1(pretrained=True)
for param in model6.features[0:12].parameters():
    param.requires_grad = False
model6.classifier = nn.Sequential(
    nn.Dropout(p=0.5, inplace=False),
    nn.Conv2d(512, 3, kernel_size=(1, 1), stride=(1, 1)),
    nn.ReLU(inplace=True),
    nn.AdaptiveAvgPool2d(output_size=(1, 1)),
    nn.Flatten(),
    )

model6.load_state_dict(torch.load("./model_params/" + "squeezenet" + "_cv_dev" + str(6) + ".pth"))

model6 = nn.Sequential(model6.features, nn.AdaptiveAvgPool2d(output_size=(1, 1)), nn.Flatten())

model6 = model6.to(device)
# ################################################################################

model7 = models.shufflenet_v2_x1_0(pretrained=True)
for param in model7.parameters():
    param.requires_grad = False
    
# for param in model2.stage4[3].parameters():
#     param.requires_grad = True
for param in model7.conv5.parameters():
    param.requires_grad = True

model7.fc = nn.Linear(in_features=1024, out_features=3, bias=True)
model7.load_state_dict(torch.load("./model_params/" + "shufflenet" + "_cv_dev" + str(7) + ".pth"))

model7 = nn.Sequential(*list(model7.children())[:-1], nn.AdaptiveAvgPool2d(output_size=(1, 1)), nn.Flatten())

model7 = model7.to(device)
# # ################################################################################
model8 = models.resnext101_32x8d(pretrained=True)

for param in model8.parameters():
    param.requires_grad = False
for param in model8.layer4[2].parameters():
    param.requires_grad = True
model8.fc = nn.Linear(2048, 3)
model8.load_state_dict(torch.load("./model_params/" + "resnext101" + "_cv_dev" + str(8) + ".pth"))

model8 = nn.Sequential(*list(model8.children())[:-1], nn.Flatten())

model8 = model8.to(device)


In [ ]:
def test_model(loader, model, which_net, which_dev, flag):
  if which_net == "vgg":
       scores_true = torch.empty([0, 512]).to(device)
  elif which_net == "squeezenet":
       scores_true = torch.empty([0, 512]).to(device)
  elif which_net == "shufflenet":
       scores_true = torch.empty([0, 1024]).to(device)
  elif which_net == "resnext101":
       scores_true = torch.empty([0, 2048]).to(device)
  labels = torch.empty(0).to(device)
  
#   model.load_state_dict(torch.load("./model_params/" + which_net + "_cv_dev" + str(which_dev) + ".pth"))


  model.eval()  # set model to evaluation mode
  with torch.no_grad():
      for data in loader:
          for x, y in data:
              x = x.to(device=device)  # move to device, e.g. GPU
              y = y.to(device=device)
              if flag == 0:
                  labels = torch.cat((labels, y))
              scores = model(x)
            #   print(scores.size())
              scores_true = torch.cat((scores_true, scores), 0)
          
      
  return scores_true, labels


In [ ]:
nets = ["vgg", "squeezenet", "shufflenet", "resnext101"]
sets = [metaset1_loader, metaset2_loader, metaset3_loader, metaset4_loader]
# models = [model1, model2, model3, model4]

metadata = torch.empty([0, 4096]).to(device)
metalabel = torch.empty([0]).to(device)
for which_dev in range(0,4):
    metaset = sets[which_dev]
    hdata = torch.empty([2800,0]).to(device)
    flag = 0
    for which_net in range(0, 4):
        net = nets[which_net]
        if which_net == 0:
            model = model5.to(device)
        elif which_net == 1:
            model = model6.to(device)
        elif which_net == 2:
            model = model7.to(device)
        elif which_net == 3:
            model = model8.to(device)

        # model = models[which_net]
        print(f'{net} with dataset{which_dev + 1}')
        scores, labels = test_model(metaset, model, net, which_dev + 1, flag)
        print(scores.size())
        hdata = torch.cat((hdata, scores), 1)
        if flag == 0:
            metalabel = torch.cat((metalabel, labels))
        flag = 1
        print()
    print()
    print()
    metadata = torch.cat((metadata, hdata), 0)
        



vgg with dataset1
torch.Size([2800, 512])

squeezenet with dataset1
torch.Size([2800, 512])

shufflenet with dataset1
torch.Size([2800, 1024])

resnext101 with dataset1
torch.Size([2800, 2048])



vgg with dataset2
torch.Size([2800, 512])

squeezenet with dataset2
torch.Size([2800, 512])

shufflenet with dataset2
torch.Size([2800, 1024])

resnext101 with dataset2
torch.Size([2800, 2048])



vgg with dataset3
torch.Size([2800, 512])

squeezenet with dataset3
torch.Size([2800, 512])

shufflenet with dataset3
torch.Size([2800, 1024])

resnext101 with dataset3
torch.Size([2800, 2048])



vgg with dataset4
torch.Size([2800, 512])

squeezenet with dataset4
torch.Size([2800, 512])

shufflenet with dataset4
torch.Size([2800, 1024])

resnext101 with dataset4
torch.Size([2800, 2048])





SyntaxError: ignored

In [ ]:
metalabel.size()

torch.Size([11200])

In [ ]:
metadata.size()

torch.Size([11200, 4096])

In [ ]:
torch.save(metadata, "dataset/metadata2")
torch.save(metalabel, "dataset/metalabel2")

In [ ]:
import seaborn as sn
df_cm = pd.DataFrame(cm.numpy().astype(np.int32),
                     index = [i for i in ["true_A", "true_B", "true_C"]],
                     columns = [i for i in ["pred_A", "pred_B", "pred_C"]])
plt.figure(figsize = (4, 4))
sn.heatmap(df_cm, annot=True, cmap="BuPu")

In [ ]:
cm.numpy().astype(np.int32)

In [ ]:
24 batch_size = 128 ok
hw:do 13
change batch_size to 128
same

deal with the bug in this file about the score